In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_path = "optimization-hashira/roberta-ai-text-detector"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [33]:
import torch    
from tqdm import tqdm
from raid import run_detection, run_evaluation
from raid.utils import load_data

train_df = load_data(split="train",include_adversarial=False)

100%|██████████| 802M/802M [00:24<00:00, 32.5MB/s]


In [34]:
train_df

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,human,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,human,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,human,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,human,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,human,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...
...,...,...,...,...,...,...,...,...,...,...,...
467980,dc125710-a42d-4403-94d1-e7e25307a6d1,dc125710-a42d-4403-94d1-e7e25307a6d1,b1e69455-73c9-47b6-a849-da38cdef929a,mpt-chat,sampling,no,none,wiki,Preferred number,"Write the body of a Wikipedia article titled ""...",Preferred number is a term used in cognitive p...
467981,a0cb4901-551b-4f64-a32b-4b9046bc7209,a0cb4901-551b-4f64-a32b-4b9046bc7209,c9b8f9f6-8c41-45b0-9509-2466f79d0346,mpt-chat,greedy,yes,none,wiki,Francisco Bethencourt,"Write the body of a Wikipedia article titled ""...",Francisco de Paula Victorino dos Santos Ferrei...
467982,85fd6cde-1e2b-4c22-a8f7-03121b6f8e8f,85fd6cde-1e2b-4c22-a8f7-03121b6f8e8f,c9b8f9f6-8c41-45b0-9509-2466f79d0346,mpt-chat,sampling,no,none,wiki,Francisco Bethencourt,"Write the body of a Wikipedia article titled ""...",Francisco Bethencourt was a remarkable Portugu...
467983,4693b180-9160-4daf-9a02-5a64f26d6e45,4693b180-9160-4daf-9a02-5a64f26d6e45,defc36fc-661e-400c-a364-03610f90bd04,mpt-chat,greedy,yes,none,wiki,Bart Allen,"Write the body of a Wikipedia article titled ""...",Bart Allen is a fictional superhero appearing ...


In [ ]:
def my_detector(texts: list[str]) -> list[float]:
    predictions = []
    for text in tqdm(texts):
        inputs = tokenizer(
            text,
            max_length=256,
            truncation=True,
            return_tensors="pt"
        ).to("cuda")
        output = model(**inputs)
        probs = output.logits.softmax(dim=-1)
        _,fake = probs.detach().cpu().flatten().numpy().tolist()
        predictions.append(fake)
    return predictions

In [36]:
predictions = run_detection(my_detector, train_df)

evaluation_result = run_evaluation(predictions, train_df)

100%|██████████| 467985/467985 [43:32<00:00, 179.10it/s]


In [37]:
evaluation_result

{'scores': [{'domain': 'abstracts',
   'model': 'llama-chat',
   'decoding': 'greedy',
   'repetition_penalty': 'no',
   'attack': 'none',
   'accuracy': {'0.05': {'tp': 20,
     'fn': 1746,
     'accuracy': 0.011325028312570781}},
   'auroc': 0.24097685102649904},
  {'domain': 'abstracts',
   'model': 'llama-chat',
   'decoding': 'greedy',
   'repetition_penalty': 'yes',
   'attack': 'none',
   'accuracy': {'0.05': {'tp': 18,
     'fn': 1748,
     'accuracy': 0.010192525481313703}},
   'auroc': 0.29425851204775233},
  {'domain': 'abstracts',
   'model': 'llama-chat',
   'decoding': 'greedy',
   'repetition_penalty': 'all',
   'attack': 'none',
   'accuracy': {'0.05': {'tp': 38,
     'fn': 3494,
     'accuracy': 0.010758776896942242}},
   'auroc': 0.2676176815371257},
  {'domain': 'abstracts',
   'model': 'llama-chat',
   'decoding': 'sampling',
   'repetition_penalty': 'no',
   'attack': 'none',
   'accuracy': {'0.05': {'tp': 26,
     'fn': 1740,
     'accuracy': 0.014722536806342015}